In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import openpyxl
from  openpyxl import *

In [54]:
def find_url_list():
    kitap = load_workbook("WebScraping.xlsm")
    url_list = []
    sheet=kitap['Links']
    for row in sheet.iter_rows(min_row=2, min_col=1, max_row=sheet.max_row, max_col=1):
        for cell in row:
            if cell.value is not None:
                url_list.append(cell.value)
    return url_list

In [53]:
def veri_cek(url): 
    page = requests.get(url)     #ilgili sayfayı python'a download edebilmek icin request modülünü kullanalım
    soup = BeautifulSoup(page.content, "html.parser")     # BeautifulSoup kütüphanesini kullanarak sitemizi bolumlerine ayıralım
    data = [] #cıktımızı elde etmek icin daha sonra kullanacagımız data listesi olusturalım
    
    uls = soup.find_all("ul") # Sayfadaki list elemanları arasında gezinebilmek icin unordered list'leri bulalım
    h3s = soup.find_all("h3") # Sayfadaki h3 dereceden basliklar arasında gezinebilmek icin h3 basliklari bulalım
    h4s = soup.find_all("h4") # Sayfadaki h4 dereceden basliklar arasında gezinebilmek icin h4 basliklari bulalım
    
    # Bazı sayfalarda hic h3 dereceden baslik olmayabiliyor, o yüzden hata fırlatmasına karşı try dongusune alalım
    # Biz sayfada General Characteristics, components ve performance başlığı altındaki listeleri istiyoruz. 
    # Ancak bu başlıklar bazı sayfalarda h3 bazı sayfalarda h4 olarak kodlanmış.
    
    try: 
        if any(soup.find(lambda tag:tag.name=="h3" and "characteristics" in tag.text)):
            
            current_baslik = h3s #Eger ilgili sayfadaki general characteristics basligi h3 derecedense o zaman h3 numaralı basliklar isimize yarayacaktır.
    except:
        current_baslik =  h4s # Eger ilgili sayfadaki general characteristics basligi h4 derecedense o zaman h4 numaralı basliklar isimize yarayacaktır.
            
            
    for header in current_baslik: #bir dongu yardımıyla ilgili baslik tipi (h3 veya h4) icinde gezinelim
        if (header.text == "General characteristics") or (header.text == "Components") or (header.text == "Performance"):
            ul = header.find_next("ul") #ilgili basliga geldik ve bir sonraki unordered listi elde etmek istiyoruz
            lis = ul.find_all("li") #ilgili unordered listedeki liste elemanlarını elde edelim 
    
            for li in lis: #elde ettigimiz liste elemanları arasında dongu dondurelim
                #Wikipedia sayfaları farklı sekilde HTML yapılarına sahip olabilir. Örn: bazı wikipedia sayfalarında "General characteristics" h3 baslik iken
                #bazı wikipedia sayfalarında h4 olarak kodlanmış. Aynı şekilde bazı wikipedia sayfalarında ilgili motor ismi Specifications başlığının yanında parantez
                #icinde yazarken, bazı sayfalarda ise ya h1 basligi olarak, ya da bir önceki h3 basligi olarak bulunmakta. Tüm wikipedia sayfaları ortak yapıda olmadığından
                #islem biraz daha karmasik hal almakta. 
                
                if "(" in li.find_previous("h2").text: #eger specifications basliginda motor ismi yazıyorsa
                    parantez_ici = li.find_previous("h2").text[li.find_previous("h2").text.find("(")+1:li.find_previous("h2").text.find(")")] #motor ismini elde ediyoruz
                    try:
                        if parantez_ici in li.find_previous("h3").text: #bazı sayfalarda parantez icinde yazan motor ismi olmayabiliyor, bu if dongusu bunu kontrol ediyor.
                            if li.find_previous("h3").find_previous("h2") != li.find_previous("h2"):
                                h3 = li.find_previous("h3")
                                h3_text = parantez_ici
                            else:
                                h3 = li.find_previous("h3")    
                                h3_text = h3.text.replace("[edit]", "") #Elde ettigimiz basliklar edit postfix'i ile gelmekte. İlgili kısmı siliyoruz.
                        else:
                            h3 = li.find_previous("h2")
                            h3_text = h3.text.replace("[edit]", "")
                            h3_text = h3_text.replace("[edit]","").split(" ",1)[1]
                            h3_text = re.sub(r'[()]', '', h3_text)
                    except:
                        #Daha önce açıkladığım gibi bazı sayfalarda h3 baslik bulunmamakta.
                        #Eger sayfada h3 basligi yoksa ve specificationun parantez icinde motor ismi yazıyorsa asagidaki donguye girer
                        
                        h3 = li.find_previous("h2")    
                        h3_text = h3.text.replace("[edit]", "") 
                        h3_text = h3_text.replace("[edit]","").split(" ",1)[1]
                        h3_text = re.sub(r'[()]', '', h3_text)
                else:
                        try:
                            h3 =  li.find_previous("h3")
                            # asagi kod prcagında eger specification ile listemiz arasında bir motor ismi yer alıyorsa onu aldırmasını saglıyoruz, 
                            # bazı durumlarda h3 basligi site'de olmasına ragmen motor ismi belirtmiyor.
                            if h3.find_previous("h2") != li.find_previous("h2"):
                                h3 = li.find_previous("h1")
                                h3_text = h3.text.replace("[edit]", "")
                            else:
                                h3_text = h3.text.replace("[edit]", "")
                        except:
                            h3 =  li.find_previous("h1")
                            h3_text = h3.text.replace("[edit]", "")
                        
                        
                h4 = li.find_previous("h4")
                try:
                    if not((h4.text == "General characteristics") or (h4.text == "Components") or (h4.text == "Performance")):
                        #print("buraya girdi")
                        h4 = li.find_previous("h3")
                except:
                    h4 = li.find_previous("h4")  
                
                if h3 and h4:
                    h4_text = h4.text.replace("[edit]", "")
                    if ":" in li.text: #Burada elde ettigimiz değerleri ":"'a göre ayırıyoruz.
                        #Bazı degerler birden fazla veri icerebilir. (Örn: maximum thrust genellikle birden fazla alt maddeden oluşuyor). Böyle olması durumunda
                        #Asagida bir if dongusu acarak ona gore islem yapmasını sağlıyorum. 
                        if "\n" in li.text.split(":",1)[1].strip(): 
                            for i in range(0,li.text.split(":",1)[1].strip().count("\n")+1):
                                data.append([h3_text, h4_text, li.text.split(":",1)[0].strip(), li.text.split(":",1)[1].strip().split("\n")[i].strip()])    
                        else: # Bazı verilerde ":" 
                            data.append([h3_text, h4_text, li.text.split(":",1)[0].strip(), li.text.split(":",1)[1].strip()])
                    #eger : yoksa asagidaki else'e girer
                    else:
                       data.append([h3_text, h4_text, "", li.text])     
                        
                        
                        
                else:
                    if h4==None:
                        h4 = li.find_previous("h3")
                    
                    h4_text = h4.text.replace("[edit]","")
                    #eger h3 baslik yoksa h2 basligin icindeki parantezi alalım,
                    try:
                        #Eger specifications basliginda parantez yoksa yani motor ismi belirtilmemişse ve h3/h4 baslik da sitede yoksa o zaman motor ismi olarak  
                        #h1 dereceden basligi atayalım diyorum.
                        h3_text = li.find_previous("h2").text.replace("[edit]","").split(" ",1)[1]
                        h3_text = re.sub(r'[()]', '', h3_text)
                    except:
                        h3_text = li.find_previous("h1").text
                    if ":" in li.text:
                        if "\n" in li.text.split(":",1)[1].strip():
                            for i in range(0,li.text.split(":",1)[1].strip().count("\n")+1):
                                data.append([h3_text, h4_text, li.text.split(":",1)[0].strip(), li.text.split(":",1)[1].strip().split("\n")[i].strip()])    
                        else:
                            data.append([h3_text, h4_text, li.text.split(":",1)[0].strip(), li.text.split(":",1)[1].strip()])
                    else:
                        data.append([h3_text, h4_text, "", li.text])     
    df = pd.DataFrame(data, columns=["Engine Type ", "Specification Name ", "Key", "Değer"]) #ilgili verilerden oluşan bir dataframe oluşturalım.
  
    return df


In [8]:
url_list = find_url_list()
df = pd.DataFrame(columns=["Engine Type ", "Specification Name ", "Key", "Değer"])
for i in range(len(url_list)):
    df = pd.concat([df,veri_cek(url_list[i])])
df

,Engine Type,Specification Name,Key,Değer
0,430D,General characteristics,Type,Direct drive two-stroke piston aircraft engine
1,430D,General characteristics,Bore,2.658 in (68 mm)
2,430D,General characteristics,Stroke,2.362 in (60 mm)
3,430D,General characteristics,Displacement,428 cc (26.11 cubic inches)
4,430D,General characteristics,Dry weight,65 lb (29 kg)
5,430D,General characteristics,Designer,McCornack/Cuyuna
6,430D,Components,Fuel type,Regular auto fuel
7,430D,Components,Oil system,40:1 premixed fuel and oil
8,430D,Components,Cooling system,fan forced air
9,430D,Performance,Power output,30 hp (22 kW)
